# OpenAI assistants | OpenAIアシスタント

> [Assistants API](https://platform.openai.com/docs/assistants/overview)を使用すると、独自のアプリケーション内でAIアシスタントを構築できます。アシスタントは指示を受け、モデル、ツール、知識を活用してユーザーの問い合わせに対応します。Assistants APIは現在、コードインタープリター、情報検索、関数呼び出しの3種類のツールをサポートしています。
>
> > The [Assistants API](https://platform.openai.com/docs/assistants/overview) allows you to build AI assistants within your own applications. An Assistant has instructions and can leverage models, tools, and knowledge to respond to user queries. The Assistants API currently supports three types of tools: Code Interpreter, Retrieval, and Function calling

OpenAIアシスタントとの対話には、OpenAIのツールまたはカスタムツールを使用することができます。OpenAIのツールのみを使用する場合は、アシスタントを直接呼び出して最終的な回答を得られます。カスタムツールを使用する場合、組み込みのAgentExecutorを利用してアシスタントとツールの実行ループを行うか、あるいは自分自身でエグゼキュータを容易に書くことが可能です。

> You can interact with OpenAI Assistants using OpenAI tools or custom tools. When using exclusively OpenAI tools, you can just invoke the assistant directly and get final answers. When using custom tools, you can run the assistant and tool execution loop using the built-in AgentExecutor or easily write your own executor.

以下では、アシスタントとのやり取りするさまざまな方法を示します。簡単な例として、コードを書いて実行できる数学のチューターを作成してみましょう。

> Below we show the different ways to interact with Assistants. As a simple example, let's build a math tutor that can write and run code.


### Using only OpenAI tools | OpenAIのツールのみを使用して


In [1]:
from langchain.agents.openai_assistant import OpenAIAssistantRunnable

In [12]:
interpreter_assistant = OpenAIAssistantRunnable.create_assistant(
    name="langchain assistant",
    instructions="You are a personal math tutor. Write and run code to answer math questions.",
    tools=[{"type": "code_interpreter"}],
    model="gpt-4-1106-preview",
)
output = interpreter_assistant.invoke({"content": "What's 10 - 4 raised to the 2.7"})
output

[ThreadMessage(id='msg_qgxkD5kvkZyl0qOaL4czPFkZ', assistant_id='asst_0T8S7CJuUa4Y4hm1PF6n62v7', content=[MessageContentText(text=Text(annotations=[], value='The result of the calculation \\(10 - 4^{2.7}\\) is approximately \\(-32.224\\).'), type='text')], created_at=1700169519, file_ids=[], metadata={}, object='thread.message', role='assistant', run_id='run_aH3ZgSWNk3vYIBQm3vpE8tr4', thread_id='thread_9K6cYfx1RBh0pOWD8SxwVWW9')]

### As a LangChain agent with arbitrary tools | 任意のツールを備えたLangChainエージェントとして

それでは、私たち自身のツールを使ってこの機能を再現してみましょう。この例では、[E2B sandbox runtime tool](https://e2b.dev/docs?ref=landing-page-get-started)を使用します。

> Now let's recreate this functionality using our own tools. For this example we'll use the [E2B sandbox runtime tool](https://e2b.dev/docs?ref=landing-page-get-started).


In [ ]:
!pip install e2b duckduckgo-search

In [3]:
import getpass

from langchain.tools import DuckDuckGoSearchRun, E2BDataAnalysisTool

tools = [E2BDataAnalysisTool(api_key=getpass.getpass()), DuckDuckGoSearchRun()]

In [4]:
agent = OpenAIAssistantRunnable.create_assistant(
    name="langchain assistant e2b tool",
    instructions="You are a personal math tutor. Write and run code to answer math questions. You can also search the internet.",
    tools=tools,
    model="gpt-4-1106-preview",
    as_agent=True,
)

#### Using AgentExecutor | AgentExecutorの使用

OpenAIAssistantRunnableはAgentExecutorと互換性があるため、エージェントとして直接エグゼキューターに渡すことが可能です。AgentExecutorは、呼び出されたツールを実行し、そのツールの出力をAssistants APIにアップロードする役割を担います。さらに、LangSmithのトレース機能が組み込まれています。

> The OpenAIAssistantRunnable is compatible with the AgentExecutor, so we can pass it in as an agent directly to the executor. The AgentExecutor handles calling the invoked tools and uploading the tool outputs back to the Assistants API. Plus it comes with built-in LangSmith tracing.


In [5]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools)
agent_executor.invoke({"content": "What's the weather in SF today divided by 2.7"})

{'content': "What's the weather in SF today divided by 2.7",
 'output': "The search results indicate that the weather in San Francisco is 67 °F. Now I will divide this temperature by 2.7 and provide you with the result. Please note that this is a mathematical operation and does not represent a meaningful physical quantity.\n\nLet's calculate 67 °F divided by 2.7.\nThe result of dividing the current temperature in San Francisco, which is 67 °F, by 2.7 is approximately 24.815.",
 'thread_id': 'thread_hcpYI0tfpB9mHa9d95W7nK2B',
 'run_id': 'run_qOuVmPXS9xlV3XNPcfP8P9W2'}

:::tip

[LangSmith トレース](https://smith.langchain.com/public/6750972b-0849-4beb-a8bb-353d424ffade/r)

> [LangSmith trace](https://smith.langchain.com/public/6750972b-0849-4beb-a8bb-353d424ffade/r)

:::


#### Custom execution | カスタム実行

また、LCELを使用すれば、アシスタントを実行するための独自の実行ループを簡単に書くことができます。これにより、実行過程を完全にコントロールすることができます。

> Or with LCEL we can easily write our own execution loop for running the assistant. This gives us full control over execution.


In [6]:
agent = OpenAIAssistantRunnable.create_assistant(
    name="langchain assistant e2b tool",
    instructions="You are a personal math tutor. Write and run code to answer math questions.",
    tools=tools,
    model="gpt-4-1106-preview",
    as_agent=True,
)

In [7]:
from langchain_core.agents import AgentFinish


def execute_agent(agent, tools, input):
    tool_map = {tool.name: tool for tool in tools}
    response = agent.invoke(input)
    while not isinstance(response, AgentFinish):
        tool_outputs = []
        for action in response:
            tool_output = tool_map[action.tool].invoke(action.tool_input)
            print(action.tool, action.tool_input, tool_output, end="\n\n")
            tool_outputs.append(
                {"output": tool_output, "tool_call_id": action.tool_call_id}
            )
        response = agent.invoke(
            {
                "tool_outputs": tool_outputs,
                "run_id": action.run_id,
                "thread_id": action.thread_id,
            }
        )

    return response

In [8]:
response = execute_agent(agent, tools, {"content": "What's 10 - 4 raised to the 2.7"})
print(response.return_values["output"])

e2b_data_analysis {'python_code': 'result = 10 - 4 ** 2.7\nprint(result)'} {"stdout": "-32.22425314473263", "stderr": "", "artifacts": []}

\( 10 - 4^{2.7} \) equals approximately -32.224.


## Using existing Thread | 既存のスレッドを使用する

既存のスレッドを使用するには、エージェントを呼び出す際に「thread\_id」を渡すだけです。

> To use an existing thread we just need to pass the "thread\_id" in when invoking the agent.


In [9]:
next_response = execute_agent(
    agent,
    tools,
    {"content": "now add 17.241", "thread_id": response.return_values["thread_id"]},
)
print(next_response.return_values["output"])

e2b_data_analysis {'python_code': 'result = 10 - 4 ** 2.7 + 17.241\nprint(result)'} {"stdout": "-14.983253144732629", "stderr": "", "artifacts": []}

\( 10 - 4^{2.7} + 17.241 \) equals approximately -14.983.


## Using existing Assistant | 既存のアシスタントを使用して

既存のアシスタントを使用するには、`assistant_id`を直接指定して`OpenAIAssistantRunnable`を初期化できます。

> To use an existing Assistant we can initialize the `OpenAIAssistantRunnable` directly with an `assistant_id`.


In [ ]:
agent = OpenAIAssistantRunnable(assistant_id="<ASSISTANT_ID>", as_agent=True)